In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Data preprocessing

In [ ]:
dataset = np.load('./data_train.npy').T
labels = np.load('./labels_train.npy')
data_train, data_val, labels_train, labels_val = train_test_split(dataset, 
                                                                    labels,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    random_state=0)
data_train

In [ ]:
data_train_size = data_train.shape[0]
data_train_shaped = np.zeros(shape=(data_train_size,300,300,3))
for image in range(data_train_size):
    one_img = data_train[image].reshape((300, 300, 3))
    data_train_shaped[image] = one_img
data_train_shaped.shape

In [ ]:
data_val_size = data_val.shape[0]
data_val_shaped = np.zeros(shape=(data_val_size,300,300,3))
for image in range(data_val_size):
    one_img = data_val[image].reshape((300, 300, 3))
    data_val_shaped[image] = one_img
data_val_shaped.shape

In [ ]:
import gc
del dataset
del labels
del data_train
del data_val
gc.collect()

In [ ]:
data_train_shaped = data_train_shaped.astype('uint8')
labels_train = labels_train.astype('uint8')
data_val_shaped = data_val_shaped.astype('uint8')
labels_val = labels_val.astype('uint8')

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train_shaped, labels_train))

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((data_val_shaped, labels_val))

In [ ]:
BATCH_SIZE = 42
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset

Show the first nine images and labels from the training set:

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(labels[i].numpy())
    plt.axis("off")

As the original dataset doesn't contain a test set, you will create one. To do so, determine how many batches of data are available in the validation set using `tf.data.experimental.cardinality`, then move 20% of them to a test set.

In [ ]:
print('Number of train batches: %d' % tf.data.experimental.cardinality(train_dataset))
print('Number of val batches: %d' % tf.data.experimental.cardinality(val_dataset))

In [ ]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
train_ds, labels_train

# Simple/NoDataAug/NoDropout

In [ ]:
num_classes = 10
model = Sequential([
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Simple/NoDataAug/Dropout

In [ ]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Simple/DataAugSimple/NoDropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Simple/DataAugSimple/Dropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Simple/DataAugAdv/NoDropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Simple/DataAugAdv/Dropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(300, 300, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# MobileNetTransfer/NoDataAug/NoDropout

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
  layers.Rescaling(1./127.5, input_shape=(300, 300, 3)),
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# MobileNetTransfer/DataAugSimple/NoDropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./127.5, input_shape=(300, 300, 3)),
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# MobileNetTransfer/DataAugSimple/Dropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./127.5, input_shape=(300, 300, 3)),
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# MobileNetTransfer/DataAugAdv/NoDropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./127.5, input_shape=(300, 300, 3)),
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# MobileNetTransfer/DataAugAdv/Dropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./127.5, input_shape=(300, 300, 3)),
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
learning_rate = 0.0003
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# EfficientNetB0/DataAugSimple/Dropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
base_model = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
  data_augmentation,
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(history.epoch[-1]+1)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True
history

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 200

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
  epochs=epochs
)

# EfficientNetB0/DataAugAdv/Dropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
base_model = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = Sequential([
  data_augmentation,
  base_model,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(10, name="outputs")
])

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(history.epoch[-1]+1)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True
history

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 200

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
  epochs=epochs
)

In [ ]:
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for images, labels in val_ds.take(29):
    print(images.shape)

In [ ]:
predictions = model.predict(val_ds)
score = tf.nn.softmax(predictions[17])
predictions.shape

In [ ]:
score
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
y_val = np.zeros(shape=predictions.shape[0])

for i in range(predictions.shape[0]):
    score = tf.nn.softmax(predictions[i])
    y_val[i] = class_names[np.argmax(score)]
    

In [ ]:
y_val = y_val.astype('uint8')

In [ ]:
y_val.shape

In [ ]:
labels_val[0]

In [ ]:
val_cm = confusion_matrix(labels_val, y_val)

In [ ]:
ConfusionMatrixDisplay(val_cm).plot()

In [ ]:
y_val.shape[0]

In [ ]:
misclassified = 0
for i in range(labels_val.shape[0]):
    if y_val[i] != labels_val[i]:
      misclassified += 1
misclassified

In [ ]:
classification = np.array(list(map(lambda x, y: x == y, y_val, labels_val)))
np.unique(classification, return_counts=True)

In [ ]:
j = 0
k = 0
plt.figure(figsize=(30, 500))
for images, labels in val_ds.take(29):
    for i in range(images.shape[0]):
      if not classification[k]:
        ax = plt.subplot(50, 4, j + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'{y_val[k]}, {labels[i]}')
        plt.axis("off")
        j += 1
      k += 1
print(j)

# EfficientNetB7/DataAugSimple/Dropout

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(300,
                                  300,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
#from tensorflow.keras.applications import EfficientNetB0

base_model = tf.keras.applications.EfficientNetB7(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential([
  data_augmentation,
  base_model,
  tf.keras.layers.Conv2D(32, 1, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 5, padding='same', activation='selu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='selu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=.0001),#.0001 or less
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [ ]:
epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(history.epoch[-1]+1)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 200

learning_rate = 0.0001

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

# EfficientNetB7/AdvDataAug/Dropout

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomFlip('vertical'),
    tf.keras.layers.RandomRotation(0.5), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.5), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.5),  # Add RandomZoom layer for a change in perspective
    tf.keras.layers.RandomFlip('horizontal', input_shape=(300, 300, 3)), #Flips Images Randomly Horizontally
    tf.keras.layers.RandomRotation(0.2), # Rotates Images +- 0.2rad (11.5deg)
    tf.keras.layers.RandomContrast(0.1), #Add contrast from .1 to 1 of original
    tf.keras.layers.RandomZoom(0.2),  # Add RandomZoom layer for a change in perspective
])

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (300, 300, 3)
#from tensorflow.keras.applications import EfficientNetB0

base_model = tf.keras.applications.EfficientNetB7(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.models.Sequential([
  data_augmentation,
  base_model,
  tf.keras.layers.Conv2D(32, 1, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='selu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 5, padding='same', activation='selu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='selu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, name="outputs")
])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=.0001),#.0001 or less
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [ ]:
epochs=300
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(history.epoch[-1]+1)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 200

learning_rate = 0.0001

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs=20
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

In [ ]:
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for images, labels in val_ds.take(29):
    print(images.shape)

In [ ]:
predictions = model.predict(val_ds)
score = tf.nn.softmax(predictions[17])
predictions.shape

In [ ]:
score
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
y_val = np.zeros(shape=predictions.shape[0])

for i in range(predictions.shape[0]):
    score = tf.nn.softmax(predictions[i])
    y_val[i] = class_names[np.argmax(score)]
    

In [ ]:
y_val = y_val.astype('uint8')

In [ ]:
y_val.shape

In [ ]:
labels_val[0]

In [ ]:
val_cm = confusion_matrix(labels_val, y_val)

In [ ]:
ConfusionMatrixDisplay(val_cm).plot()

In [ ]:
y_val.shape[0]

In [ ]:
misclassified = 0
for i in range(labels_val.shape[0]):
    if y_val[i] != labels_val[i]:
      misclassified += 1
misclassified

In [ ]:
classification = np.array(list(map(lambda x, y: x == y, y_val, labels_val)))
np.unique(classification, return_counts=True)

In [ ]:
j = 0
k = 0
plt.figure(figsize=(30, 500))
for images, labels in val_ds.take(29):
    for i in range(images.shape[0]):
      if not classification[k]:
        ax = plt.subplot(50, 4, j + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'{y_val[k]}, {labels[i]}')
        plt.axis("off")
        j += 1
      k += 1
print(j)

# Fake Test

In [ ]:
data_test = np.load('./data.npy').T
labels_test = np.load('./labels.npy')
data_test.shape, labels_test.shape

In [ ]:
data_test_size = data_test.shape[0]
data_test_shaped = np.zeros(shape=(data_test_size,300,300,3))
for image in range(data_test_size):
    one_img = data_test[image].reshape((300, 300, 3))
    data_test_shaped[image] = one_img
data_test_shaped.shape

In [ ]:
labels_val = labels_val.astype('uint8')

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((data_test_shaped, labels_test))

In [ ]:
BATCH_SIZE = 42

test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

test_ds = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_ds, labels_test

In [ ]:
model.predict(test_ds)

In [ ]:
class_names = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
for images, labels in test_ds.take(8):
    print(images.shape)

In [ ]:
predictions = model.predict(test_ds)
score = tf.nn.softmax(predictions[0])
predictions

In [ ]:
score
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
y_test = np.zeros(shape=predictions.shape[0])

for i in range(predictions.shape[0]):
    score = tf.nn.softmax(predictions[i])
    y_test[i] = class_names[np.argmax(score)]
    

In [ ]:
y_test[0]

In [ ]:
labels_test[0]

In [ ]:
val_cm = confusion_matrix(labels_test, y_test)

In [ ]:
ConfusionMatrixDisplay(val_cm).plot()

In [ ]:
y_test.shape[0]

In [ ]:
misclassified = 0
for i in range(labels_test.shape[0]):
    if y_test[i] != labels_test[i]:
      misclassified += 1
misclassified

In [ ]:
classification = np.array(list(map(lambda x, y: x == y, y_test, labels_test)))
np.unique(classification, return_counts=True)

In [ ]:
j = 0
k = 0
plt.figure(figsize=(30, 500))
for images, labels in test_ds.take(29):
    for i in range(images.shape[0]):
      if not classification[k]:
        ax = plt.subplot(50, 4, j + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f'{y_test[k]}, {labels[i]}')
        plt.axis("off")
        j += 1
      k += 1
print(j)

In [ ]:
print(classification_report(labels_test, y_test))